In [1]:
from model_management.Predict import main_predict, predict_power_features
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import datetime

In [2]:
data_path = './historical/data/'

In [3]:
#model_path = './trained_model_parameters/models_tobe_evaluated/model_2024-09-14_NP/'
model_path = './trained_model_parameters/models_tobe_evaluated/model_2024-09-05_NP/'
#model_path = './trained_model_parameters/latest_model/'
main_predict(data_path=data_path, model_path=model_path,
             predict_days=60, save_prediction=False, update_prediction=True, avoid_training_set=True)

,日期,風力,太陽能,尖峰負載
0,2024-09-06,17.714434,639.150955,3986.434814
1,2024-09-07,19.271914,566.158657,3508.847900
2,2024-09-08,12.701839,0.000000,3245.462402
3,2024-09-09,14.704983,654.946751,3837.485107
4,2024-09-10,14.492647,682.581877,3833.763184
5,2024-09-11,19.234949,722.007395,3899.410889
6,2024-09-12,23.558590,559.357654,3904.704102
7,2024-09-13,20.621914,591.431687,3899.827393
8,2024-09-14,18.631168,542.393098,3415.206787
9,2024-09-15,13.821812,0.000000,3233.617432


In [4]:
weather_df = pd.read_csv(data_path + 'weather/finalized/big_table.csv')
weather_df['日期'] = pd.to_datetime(weather_df['日期'])
weather_df = weather_df[weather_df['日期'] >= datetime.datetime(2024,9,1)]
#predict_power_features
power_truth_df = pd.read_csv(data_path + 'power/power_generation_data.csv')
power_truth_df['日期'] = pd.to_datetime(power_truth_df['日期'])
power_truth_df = power_truth_df[power_truth_df['日期'] >= datetime.datetime(2024,9,1)]

In [5]:
#model_path = './trained_model_parameters/models_tobe_evaluated/model_2024-09-05_NP/'
df_acc_weather = predict_power_features(model_path=model_path, input_weather_df=weather_df)
df_pratical_predict = main_predict(data_path=data_path, model_path=model_path, predict_days=8, save_prediction=False, update_prediction=True, avoid_training_set=True)
df_acc_weather['日期'] = pd.to_datetime(df_acc_weather['日期'])
df_pratical_predict['日期'] = pd.to_datetime(df_pratical_predict['日期'])
df_preds = pd.merge(df_acc_weather, df_pratical_predict, on='日期', suffixes=['_base_on_obs', '_base_on_forecast'])
df_compare = pd.merge(df_preds, power_truth_df[['日期', '風力發電', '太陽能發電', '尖峰負載']], on='日期', suffixes=['', '_truth'])
df_compare

,日期,風力_base_on_obs,太陽能_base_on_obs,尖峰負載_base_on_obs,風力_base_on_forecast,太陽能_base_on_forecast,尖峰負載_base_on_forecast,風力發電,太陽能發電,尖峰負載
0,2024-09-16,50.901409,698.280251,3865.955078,34.751854,706.689527,3917.334473,23.71,704.0451,36822.451
1,2024-09-17,45.944942,813.261588,3446.125732,16.932846,718.499626,3401.590088,41.82,808.9387,33349.287
2,2024-09-18,144.104950,624.849939,3833.921875,94.436607,672.665742,3913.409668,250.67,558.2749,38477.649
3,2024-09-19,50.402233,685.849549,3925.137207,60.667278,591.663717,3919.045654,13.62,678.1897,39019.697
4,2024-09-20,117.131271,497.176995,3807.098633,50.632736,527.123462,3853.908447,49.49,550.3974,38546.774
5,2024-09-21,18.325918,425.185442,3203.455566,62.944038,458.379590,3412.663330,14.28,285.2004,32479.404
6,2024-09-22,171.359772,0.000000,3018.167725,124.656242,0.000000,3074.278320,169.45,0.0000,29872.500


In [6]:
df = pd.merge(power_truth_df, df_acc_weather, on='日期')
print(np.mean(np.abs(df['風力'] - df['風力發電'])))

29.07523311788386


In [7]:
df = pd.merge(power_truth_df, df_pratical_predict, on='日期')
print(np.mean(np.abs(df['風力'] - df['風力發電'])))

47.68717300415039
